In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set up visualization style
plt.style.use('ggplot')
sns.set_palette("Set2")
%matplotlib inline

# Check if we have data
data_path = Path('../data/raw')
print("Available data files:")
for file in data_path.glob('*'):
    print(f"  - {file.name}")